In [17]:
import datetime
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings
import sklearn
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 400


from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import KFold, StratifiedKFold

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

FEATS_EXCLUDED = ['target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0', 'hist_purchase_date_ptp', 'new_purchase_date_ptp'
                 ]

DEFAULT_PARAMS = {
                'task': 'train',
                'boosting': 'gbdt',
                'objective': 'regression',
                'metric': 'rmse',
                #'learning_rate': 0.0011282422952476733,
                'learning_rate': 0.01,
                 'feature_fraction': 0.352298986480031,
                 'num_leaves': 99,
                 'min_data_in_leaf': 21,
                 'max_depth': 8,
                 'reg_alpha': 27.736776301810206,
                 'reg_lambda': 19.2548648172899,
                 'subsample': 0.8155675399742167,
                 'colsample_bytree': 0.7913590364000926,
                 'min_child_weight': 10,
                 'min_split_gain': 16,
                 'bagging_freq': 8,
                 'bagging_fraction': 0.7720057646521765,
                'verbose': -1,
                'n_jobs': -1
                }

CATEGORICAL_FEATURES = ['feature_1_orig', 'feature_2_orig', 'feature_3_orig','first_active_year', 'first_active_month', 'first_active_year', 'first_active_quarter']

ADD_CATEGORICAL = [
                     'month_lag_hist_1',
                     'month_lag_hist_2',
                     'month_lag_hist_3',
                     'month_lag_hist_4',
                     'month_lag_hist_5',
                     'month_lag_hist_6',
                     'month_lag_hist_7',
                     'month_lag_hist_8',
                     'month_lag_hist_9',
                     'month_lag_hist_10',
                     'month_lag_hist_11',
                     'month_lag_hist_12',
                     'month_lag_hist_13',
                     'month_lag_new_1',
                     'month_lag_new_2']

import optuna
import xgboost as xgb
num_rows = None


def kfold_lgb_rmse(train_df, test_df, params=DEFAULT_PARAMS, num_folds=4, stratified = True, shuffle=True, debug= False):
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    
    #categorical_features = CATEGORICAL_FEATURES + ADD_CATEGORICAL
    categorical_features = CATEGORICAL_FEATURES
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=shuffle, random_state=326)
    else:
        folds = KFold(n_splits= num_folds, shuffle=shuffle, random_state=326)
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['outliers'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['target'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['target'].iloc[valid_idx]
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                categorical_feature=categorical_features)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               categorical_feature=categorical_features)

        params['seed'] = int(2**n_fold),
        params['bagging_seed'] = int(2**n_fold),
        params['drop_seed'] = int(2**n_fold)

        model = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=20000,
                        early_stopping_rounds= 500,
                        verbose_eval=100
                        )

        oof_preds[valid_idx] = model.predict(valid_x, num_iteration=model.best_iteration)
                
        sub_preds += model.predict(test_df[feats], num_iteration=model.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(model.feature_importance(importance_type='gain', iteration=model.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del model, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('RMSE is {}'.format(rmse(oof_preds, train_df['target'])))
    return sub_preds, feature_importance_df

def Mode(x):
    # if the most frequent & 2nd most frequent one have the same frequencies, take the bigger one
    counts = Counter(x)
    tup = sorted(counts.items(), key=lambda kv: kv[1])
    return tup[-1][0]

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

def add_suffix(df, num, hist=True):
    if hist:
        string = '_hist_' + str(num)
    else:
        string = '_new_' + str(num)
    new_index = df.columns + string
    df.columns = new_index

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]

    plt.figure(figsize=(12, 90))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    return best_features

In [2]:
merchants = pd.read_csv('/Users/jincui/Downloads/ELO/merchants.csv', nrows=num_rows)
merchants['category_1'] = merchants['category_1'].map({'Y': 1, 'N': 0}).astype(int)
merchants['category_4'] = merchants['category_4'].map({'Y': 1, 'N': 0}).astype(int)
merchants['category_2'][merchants['category_2'].isnull()] = -1
merchants['category_2'] = (merchants['category_2']).astype(int)
merchants['most_recent_sales_range'] = merchants['most_recent_sales_range'].map({'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}).astype(int)
merchants['most_recent_purchases_range'] = merchants['most_recent_purchases_range'].map({'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}).astype(int)
merchants = merchants.set_index('merchant_id')
merchants = merchants.add_prefix('mer_')
merchants['mer_numerical_ratio'] = merchants['mer_numerical_2'] / merchants['mer_numerical_1']
merchants.head()


,mer_merchant_group_id,mer_merchant_category_id,mer_subsector_id,mer_numerical_1,mer_numerical_2,mer_category_1,mer_most_recent_sales_range,mer_most_recent_purchases_range,mer_avg_sales_lag3,mer_avg_purchases_lag3,mer_active_months_lag3,mer_avg_sales_lag6,mer_avg_purchases_lag6,mer_active_months_lag6,mer_avg_sales_lag12,mer_avg_purchases_lag12,mer_active_months_lag12,mer_category_4,mer_city_id,mer_state_id,mer_category_2,mer_numerical_ratio
merchant_id,,,,,,,,,,,,,,,,,,,,,,
M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,0,5,5,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,0,242,9,1,1.0
M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,0,5,5,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,0,22,16,1,1.0
M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,0,5,5,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,0,-1,5,5,1.0
M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,1,5,5,NaN,1.666667,3,NaN,4.666667,6,NaN,3.833333,12,1,-1,-1,-1,1.0
M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,1,5,5,NaN,0.500000,3,NaN,0.361111,6,NaN,0.347222,12,1,-1,-1,-1,1.0


In [3]:
selected_features = ['mer_numerical_1', 'mer_numerical_2', 'mer_numerical_ratio','mer_category_1', 'mer_category_2',
           'mer_category_4', 'mer_most_recent_sales_range', 'mer_most_recent_purchases_range',
           'mer_active_months_lag3', 'mer_active_months_lag6', 'mer_active_months_lag12']

merchants[selected_features].head()



,mer_numerical_1,mer_numerical_2,mer_numerical_ratio,mer_category_1,mer_category_2,mer_category_4,mer_most_recent_sales_range,mer_most_recent_purchases_range,mer_active_months_lag3,mer_active_months_lag6,mer_active_months_lag12
merchant_id,,,,,,,,,,,
M_ID_838061e48c,-0.057471,-0.057471,1.0,0,1,0,5,5,3,6,12
M_ID_9339d880ad,-0.057471,-0.057471,1.0,0,1,0,5,5,3,6,12
M_ID_e726bbae1e,-0.057471,-0.057471,1.0,0,5,0,5,5,2,2,2
M_ID_a70e9c5f81,-0.057471,-0.057471,1.0,1,-1,1,5,5,3,6,12
M_ID_64456c37ce,-0.057471,-0.057471,1.0,1,-1,1,5,5,3,6,12


In [4]:
hist_df = pd.read_csv('/Users/jincui/Downloads/ELO/historical_transactions.csv', )
new_merchant_df = pd.read_csv('/Users/jincui/Downloads/ELO/new_merchant_transactions.csv', )
for df in [hist_df, new_merchant_df]:
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    df['purchase_amount'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['installments'].replace(-1, np.nan,inplace=True)
    df['installments'].replace(999, np.nan,inplace=True)
    df['authorized_flag'] = df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_3'] = df['category_3'].map({'A':0, 'B':1, 'C':2}).astype(int)

hist_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0.0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0.0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,1,C_ID_4e6213e9bc,88,0,0.0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,1,C_ID_4e6213e9bc,88,0,0.0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,1,C_ID_4e6213e9bc,88,0,0.0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
hist_df = hist_df.set_index('merchant_id')
new_merchant_df = new_merchant_df.set_index('merchant_id')
hist_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
merchant_id,,,,,,,,,,,,,
M_ID_e020e9b302,1,C_ID_4e6213e9bc,88,0,0.0,0,80,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
M_ID_86ec983688,1,C_ID_4e6213e9bc,88,0,0.0,0,367,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
M_ID_979ed661fc,1,C_ID_4e6213e9bc,88,0,0.0,0,80,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
M_ID_e6d5ae8ea6,1,C_ID_4e6213e9bc,88,0,0.0,0,560,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
M_ID_e020e9b302,1,C_ID_4e6213e9bc,88,0,0.0,0,80,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [6]:
hist_df = hist_df.join(merchants[selected_features], how='left')
new_merchant_df = new_merchant_df.join(merchants[selected_features], how='left')

In [7]:
hist_df = hist_df.reset_index()
new_merchant_df = new_merchant_df.reset_index()
hist_df.head()

,merchant_id,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,mer_numerical_1,mer_numerical_2,mer_numerical_ratio,mer_category_1,mer_category_2,mer_category_4,mer_most_recent_sales_range,mer_most_recent_purchases_range,mer_active_months_lag3,mer_active_months_lag6,mer_active_months_lag12
0,M_ID_000025127f,1,C_ID_6b8ec38fa5,69,0,1.0,1,80,-7,-0.728876,2017-07-22 12:43:22,1.0,9,37,-0.057471,-0.057471,1.0,0,1,1,5,5,3,6,12
1,M_ID_000025127f,1,C_ID_1ec241e2f6,69,0,0.0,0,80,-2,-0.719860,2017-12-14 16:07:26,1.0,9,37,-0.057471,-0.057471,1.0,0,1,1,5,5,3,6,12
2,M_ID_000025127f,1,C_ID_0823692398,69,0,0.0,0,80,-12,-0.734887,2017-02-17 15:30:06,1.0,9,37,-0.057471,-0.057471,1.0,0,1,1,5,5,3,6,12
3,M_ID_000025127f,1,C_ID_c636d48d1c,69,0,0.0,0,80,0,-0.738643,2018-02-23 19:12:56,1.0,9,37,-0.057471,-0.057471,1.0,0,1,1,5,5,3,6,12
4,M_ID_000025127f,1,C_ID_2957558a3d,69,0,0.0,0,80,-2,-0.728876,2017-12-23 10:58:29,1.0,9,37,-0.057471,-0.057471,1.0,0,1,1,5,5,3,6,12


In [289]:
hist_df['mer_numerical_diff'] = hist_df['mer_numerical_2'] - hist_df['mer_numerical_1']
new_merchant_df['mer_numerical_diff'] = new_merchant_df['mer_numerical_2'] - new_merchant_df['mer_numerical_1']



## Starting this step comes the experimenting

In [479]:
%%time
## seems like installments is a pretty bad feature to aggregate


aggs = {}
aggs['merchant_id'] = ['count','nunique']
aggs['purchase_amount'] = ['sum','mean']
aggs['authorized_flag'] = ['sum']
aggs['state_id'] = ['nunique']

##aggs['mer_numerical_1'] = ['median']
#aggs['mer_numerical_2'] = ['median']
##aggs['mer_numerical_diff'] = ['max', 'min']

##aggs['mer_numerical_ratio'] = ['mean']
##aggs['installments'] = ['max', 'min']
##aggs['category_1'] = ['sum']
##aggs['city_id'] = ['nunique']
##aggs['subsector_id'] = ['nunique']

##aggs['merchant_category_id'] = ['nunique']
##aggs['category_2'] = ['nunique']
##aggs['category_3'] = ['nunique']

##aggs['mer_most_recent_sales_range'] = ['max', 'min']
##aggs['mer_most_recent_purchases_range'] = ['max', 'min', 'median']
hist_agg = hist_df.groupby(['card_id', 'month_lag']).agg(aggs)
new_agg = new_merchant_df.groupby(['card_id', 'month_lag']).agg(aggs)
print(hist_agg.shape)
print(new_agg.shape)

(2563765, 6)
(480363, 6)
CPU times: user 57.7 s, sys: 2.81 s, total: 1min
Wall time: 1min


In [480]:
hist_agg.columns = ['_'.join(col).strip() for col in hist_agg.columns.values]
new_agg.columns = ['_'.join(col).strip() for col in new_agg.columns.values]
hist_agg = hist_agg.reset_index()
new_agg = new_agg.reset_index()
hist_agg = hist_agg.set_index('card_id')
new_agg = new_agg.set_index('card_id')
print(hist_agg.shape)
print(new_agg.shape)

(2563765, 7)
(480363, 7)


In [481]:
hist_agg_0 = hist_agg[hist_agg['month_lag']==0]
hist_agg_1 = hist_agg[hist_agg['month_lag']==-1]
hist_agg_2 = hist_agg[hist_agg['month_lag']==-2]
hist_agg_3 = hist_agg[hist_agg['month_lag']==-3]
hist_agg_4 = hist_agg[hist_agg['month_lag']==-4]
hist_agg_5 = hist_agg[hist_agg['month_lag']==-5]
hist_agg_6 = hist_agg[hist_agg['month_lag']==-6]
hist_agg_7 = hist_agg[hist_agg['month_lag']==-7]
hist_agg_8 = hist_agg[hist_agg['month_lag']==-8]
hist_agg_9 = hist_agg[hist_agg['month_lag']==-9]
hist_agg_10 = hist_agg[hist_agg['month_lag']==-10]
hist_agg_11 = hist_agg[hist_agg['month_lag']==-11]
hist_agg_12 = hist_agg[hist_agg['month_lag']==-12]
hist_agg_13 = hist_agg[hist_agg['month_lag']==-13]

new_agg_1 = new_agg[new_agg['month_lag']==1]
new_agg_2 = new_agg[new_agg['month_lag']==2]

num=0
for df in [hist_agg_0, hist_agg_1, hist_agg_2, hist_agg_3, hist_agg_4, hist_agg_5, hist_agg_6,
           hist_agg_7, hist_agg_8, hist_agg_9, hist_agg_10, hist_agg_11, hist_agg_12, hist_agg_13]:
    add_suffix(df, num)
    num+=1

add_suffix(new_agg_1, 1, hist=False)
add_suffix(new_agg_2, 2, hist=False)


In [482]:
train_df = pd.read_hdf('Elo_result.h5', 'train_LB_3688')
test_df = pd.read_hdf('Elo_result.h5', 'test_LB_3688')

df = train_df.append(test_df)
df.shape

(325540, 274)

In [483]:
#%%time
## just set up a standard, learning_rate is set at 0.01 intentionally to speed up training
#_, __ = kfold_lgb_rmse(train_df, test_df)

In [484]:
for DF in [hist_agg_0, hist_agg_1, hist_agg_2, hist_agg_3, hist_agg_4, hist_agg_5, hist_agg_6,
           hist_agg_7, hist_agg_8, hist_agg_9, hist_agg_10, hist_agg_11, hist_agg_12, hist_agg_13,
           new_agg_1, new_agg_2]:
    df = df.join(DF.iloc[:,1:], how='left')
df.shape


(325540, 370)

In [485]:
for col in df.columns:
    if col in ['target', 'outliers']:
        pass
    elif (df[col].isna().sum() > 0):
        if (df[col].dtypes == 'float'):
            df[col] = df[col].fillna(0.0)
        elif (df[col].dtypes == 'int'):
            df[col] = df[col].fillna(0)
        else:
            pass
train_df = df[df['target'].notnull()]
test_df = df[df['target'].isnull()]
train_df.shape

(201917, 370)

In [486]:
%%time
cv=3.6446
sub_pred, feature_importance_df = kfold_lgb_rmse(train_df, test_df)

Starting LightGBM. Train shape: (201917, 370), test shape: (123623, 370)


/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[100]	train's rmse: 3.64058	test's rmse: 3.69935
[200]	train's rmse: 3.53731	test's rmse: 3.65834
[300]	train's rmse: 3.46765	test's rmse: 3.64354
[400]	train's rmse: 3.41613	test's rmse: 3.63633
[500]	train's rmse: 3.37227	test's rmse: 3.63271
[600]	train's rmse: 3.3318	test's rmse: 3.62969
[700]	train's rmse: 3.29483	test's rmse: 3.62847
[800]	train's rmse: 3.26512	test's rmse: 3.62796
[900]	train's rmse: 3.23531	test's rmse: 3.62734
[1000]	train's rmse: 3.2062	test's rmse: 3.62665
[1100]	train's rmse: 3.17767	test's rmse: 3.62661
[1200]	train's rmse: 3.14839	test's rmse: 3.62632
[1300]	train's rmse: 3.12275	test's rmse: 3.62606
[1400]	train's rmse: 3.09984	test's rmse: 3.62656
[1500]	train's rmse: 3.07597	test's rmse: 3.62731
[1600]	train's rmse: 3.05432	test's rmse: 3.62773
[1700]	train's rmse: 3.03313	test's rmse: 3.62793
[1800]	train's rmse: 3.01428	test's rmse: 3.62843
Early stopping, best iteration is:
[1308]	train'

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[100]	train's rmse: 3.63242	test's rmse: 3.71175
[200]	train's rmse: 3.52587	test's rmse: 3.6777
[300]	train's rmse: 3.45525	test's rmse: 3.66603
[400]	train's rmse: 3.40296	test's rmse: 3.66042
[500]	train's rmse: 3.35629	test's rmse: 3.65836
[600]	train's rmse: 3.31791	test's rmse: 3.65677
[700]	train's rmse: 3.28121	test's rmse: 3.65614
[800]	train's rmse: 3.25122	test's rmse: 3.65589
[900]	train's rmse: 3.22172	test's rmse: 3.65599
[1000]	train's rmse: 3.19222	test's rmse: 3.65694
[1100]	train's rmse: 3.16342	test's rmse: 3.65753
[1200]	train's rmse: 3.13954	test's rmse: 3.65822
[1300]	train's rmse: 3.11561	test's rmse: 3.65861
Early stopping, best iteration is:
[824]	train's rmse: 3.24436	test's rmse: 3.65575
Fold  2 RMSE : 3.655750


/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[100]	train's rmse: 3.63378	test's rmse: 3.71042
[200]	train's rmse: 3.52802	test's rmse: 3.67478
[300]	train's rmse: 3.45761	test's rmse: 3.66143
[400]	train's rmse: 3.40602	test's rmse: 3.65529
[500]	train's rmse: 3.36206	test's rmse: 3.65231
[600]	train's rmse: 3.3207	test's rmse: 3.65092
[700]	train's rmse: 3.28169	test's rmse: 3.65024
[800]	train's rmse: 3.24866	test's rmse: 3.65002
[900]	train's rmse: 3.21811	test's rmse: 3.65023
[1000]	train's rmse: 3.18895	test's rmse: 3.64979
[1100]	train's rmse: 3.16143	test's rmse: 3.65031
[1200]	train's rmse: 3.13661	test's rmse: 3.65069
[1300]	train's rmse: 3.11419	test's rmse: 3.65094
[1400]	train's rmse: 3.08832	test's rmse: 3.65104
[1500]	train's rmse: 3.06528	test's rmse: 3.65121
Early stopping, best iteration is:
[1001]	train's rmse: 3.18867	test's rmse: 3.64974
Fold  3 RMSE : 3.649739


/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[100]	train's rmse: 3.63477	test's rmse: 3.70604
[200]	train's rmse: 3.53001	test's rmse: 3.66732
[300]	train's rmse: 3.45777	test's rmse: 3.65532
[400]	train's rmse: 3.40406	test's rmse: 3.65056
[500]	train's rmse: 3.35951	test's rmse: 3.6479
[600]	train's rmse: 3.31786	test's rmse: 3.64737
[700]	train's rmse: 3.28375	test's rmse: 3.64698
[800]	train's rmse: 3.25334	test's rmse: 3.6472
[900]	train's rmse: 3.22284	test's rmse: 3.64754
[1000]	train's rmse: 3.19216	test's rmse: 3.64766
[1100]	train's rmse: 3.16423	test's rmse: 3.64834
Early stopping, best iteration is:
[697]	train's rmse: 3.28482	test's rmse: 3.64691
Fold  4 RMSE : 3.646912
RMSE is 3.644606237900383
CPU times: user 34min 37s, sys: 49.1 s, total: 35min 26s
Wall time: 3min 39s


In [488]:
train_df.to_hdf('Elo_result.h5', 'train_CV_36446')
test_df.to_hdf('Elo_result.h5', 'train_CV_36446')


In [489]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

def objective(trial, num_folds=11, stratified=True):
    categorical_features = ['feature_1_orig', 'feature_2_orig', 'feature_3_orig','first_active_year', 'first_active_month', 'first_active_year', 'first_active_quarter']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=326)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=326)
        
    oof_preds = np.zeros(train_df.shape[0])
    feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['outliers'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['target'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['target'].iloc[valid_idx]

        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                categorical_feature=categorical_features)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               categorical_feature=categorical_features)
        param = {'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'n_jobs': -1, 
                 'boosting_type': 'gbdt',
                 'seed':int(2**n_fold), 'drop_seed': int(2**n_fold), 'feature_fraction_seed': int(2**n_fold),
                 'learning_rate': trial.suggest_uniform('learning_rate', 1e-3, 2e-3),
                 'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 0.6),
                 'num_leaves': trial.suggest_int('num_leaves', 60, 120),
                 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 40),
                 'max_depth': trial.suggest_int('max_depth', 6,12),
                 'reg_alpha': trial.suggest_uniform('reg_alpha', 25, 50),
                 'reg_lambda': trial.suggest_uniform('reg_lambda', 10, 30),
                 'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
                 'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
                 'min_child_weight': trial.suggest_int('min_child_weight', 10, 50),
                 'min_split_gain': trial.suggest_int('min_split_gain', 10, 50),
                }
        param['bagging_freq'] = trial.suggest_int('bagging_freq', 6, 10)
        param['bagging_fraction'] = trial.suggest_uniform('bagging_fraction', 0.5, 1.0)
        param['bagging_seed'] = 11
        param['bagging_seed'] = int(2**n_fold)
    
        clf = lgb.train(param, lgb_train, 40000, valid_sets=[lgb_train, lgb_test],
                        verbose_eval=500, early_stopping_rounds=500)
            
        oof_preds[valid_idx] = clf.predict(train_df.iloc[valid_idx][feats], num_iteration=clf.best_iteration)
        
        del clf, train_x, train_y, valid_x, valid_y, train_idx, valid_idx
        gc.collect()
    
    return rmse(oof_preds, train_df['target'])


In [490]:
%%time
study = optuna.create_study()
study.optimize(objective, n_trials=75)
print('study.best_params: \t')
print(study.best_params)
print('study.best_value: \t')
print(study.best_value)
print('study.best_trial.user_attrs: \t')
print(study.best_trial.user_attrs)

optuna_lgb = study.trials_dataframe()
optuna_lgb.to_csv("optuna_lgb_2.csv")

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 3.69384	valid_1's rmse: 3.70493
[1000]	training's rmse: 3.63015	valid_1's rmse: 3.6588
[1500]	training's rmse: 3.59463	valid_1's rmse: 3.63936
[2000]	training's rmse: 3.56962	valid_1's rmse: 3.62898
[2500]	training's rmse: 3.5492	valid_1's rmse: 3.62276
[3000]	training's rmse: 3.53238	valid_1's rmse: 3.61857
[3500]	training's rmse: 3.51695	valid_1's rmse: 3.61565
[4000]	training's rmse: 3.5039	valid_1's rmse: 3.61339
[4500]	training's rmse: 3.49181	valid_1's rmse: 3.61184
[5000]	training's rmse: 3.48129	valid_1's rmse: 3.61059
[5500]	training's rmse: 3.47178	valid_1's rmse: 3.60964
[6000]	training's rmse: 3.46285	valid_1's rmse: 3.609
[6500]	training's rmse: 3.45441	valid_1's rmse: 3.60872
[7000]	training's rmse: 3.44633	valid_1's rmse: 3.60843
[7500]	training's rmse: 3.43889	valid_1's rmse: 3.60821
[8000]	training's rmse: 3.43194	valid_1's rmse: 3.60801
[8500]	training's rmse: 3.42549	valid_1's rmse:

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 3.69211	valid_1's rmse: 3.712
[1000]	training's rmse: 3.62848	valid_1's rmse: 3.67353
[1500]	training's rmse: 3.59208	valid_1's rmse: 3.65794
[2000]	training's rmse: 3.56682	valid_1's rmse: 3.64959
[2500]	training's rmse: 3.54612	valid_1's rmse: 3.64499
[3000]	training's rmse: 3.52807	valid_1's rmse: 3.64228
[3500]	training's rmse: 3.51256	valid_1's rmse: 3.64077
[4000]	training's rmse: 3.49898	valid_1's rmse: 3.63952
[4500]	training's rmse: 3.48678	valid_1's rmse: 3.63849
[5000]	training's rmse: 3.47563	valid_1's rmse: 3.63759
[5500]	training's rmse: 3.46547	valid_1's rmse: 3.63717
[6000]	training's rmse: 3.45608	valid_1's rmse: 3.63685
[6500]	training's rmse: 3.44768	valid_1's rmse: 3.63641
[7000]	training's rmse: 3.43959	valid_1's rmse: 3.63597
[7500]	training's rmse: 3.43213	valid_1's rmse: 3.63574
[8000]	training's rmse: 3.42568	valid_1's rmse: 3.63554
[8500]	training's rmse: 3.41946	valid_1's rm

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



...

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 3.69069	valid_1's rmse: 3.72202
[1000]	training's rmse: 3.62656	valid_1's rmse: 3.68785
[1500]	training's rmse: 3.58961	valid_1's rmse: 3.67532
[2000]	training's rmse: 3.5635	valid_1's rmse: 3.66931
[2500]	training's rmse: 3.54246	valid_1's rmse: 3.66628
[3000]	training's rmse: 3.52344	valid_1's rmse: 3.66467
[3500]	training's rmse: 3.50622	valid_1's rmse: 3.66371
[4000]	training's rmse: 3.49087	valid_1's rmse: 3.66324
[4500]	training's rmse: 3.47591	valid_1's rmse: 3.663
[5000]	training's rmse: 3.46192	valid_1's rmse: 3.66278
[5500]	training's rmse: 3.44841	valid_1's rmse: 3.66293
Early stopping, best iteration is:
[5162]	training's rmse: 3.45778	valid_1's rmse: 3.66266


/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning:

Using categorical_feature in Dataset.

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 3.69198	valid_1's rmse: 3.72066
[1000]	training's rmse: 3.62839	valid_1's rmse: 3.68035
[1500]	training's rmse: 3.59158	valid_1's rmse: 3.6654
[2000]	training's rmse: 3.56588	valid_1's rmse: 3.65773
[2500]	training's rmse: 3.54437	valid_1's rmse: 3.6527
[3000]	training's rmse: 3.52586	valid_1's rmse: 3.64927
[3500]	training's rmse: 3.50921	valid_1's rmse: 3.64692
[4000]	training's rmse: 3.49451	valid_1's rmse: 3.64552


KeyboardInterrupt: 

In [ ]:
best_features = display_importances(feature_importance_df)

In [ ]:
excl_features = [*best_features['feature'][best_features['importance']==0]]
len(excl_features)
## duplicates included

In [ ]:
%%time
train_df_s = train_df.drop(excl_features, axis=1)
test_df_s = test_df.drop(excl_features, axis=1)

_, feature_importance_df_s = kfold_lgb_rmse(train_df_s, test_df_s)

In [ ]:
best_features_s = display_importances(feature_importance_df_s)

In [ ]:
#best_features_s = best_features.set_index('feature')
avg_importance = best_features_s.groupby('feature').agg({'importance': 'mean'})
avg_importance.columns = ['importance']
avg_importance = avg_importance.sort_values('importance', ascending=False)

excl = [*avg_importance.index[500:]]

In [ ]:
excl = []
for col in train_df.columns:
    if ('_hist_12' in col) or ('_hist_13' in col):
        excl.append(col)

In [ ]:
train_df_s = train_df.drop(excl, axis=1)
test_df_s = test_df.drop(excl, axis=1)
train_df_s.shape

In [ ]:
train_df = pd.read_hdf('Elo_result.h5', 'train_CV_3644')
test_df = pd.read_hdf('Elo_result.h5', 'test_CV_3644')


In [ ]:
for df in [train_df, test_df]:
    for col in df.columns:
        if col in ['target', 'outliers']:
            pass
        elif (df[col].isna().sum() > 0):
            if (df[col].dtypes == 'float'):
                df[col] = df[col].fillna(0.0)
            elif (df[col].dtypes == 'int'):
                df[col] = df[col].fillna(0)
            else:
                pass

In [ ]:
#train_df['new_purchase_date_ptp'].fillna(0, inplace=True)
#test_df['new_purchase_date_ptp'].fillna(0, inplace=True)
### These two columns have negative effect on the model
train_df = train_df.drop(['month_lag_hist_0', 'month_lag_hist_13'],axis=1)
test_df = test_df.drop(['month_lag_hist_0', 'month_lag_hist_13'],axis=1)

In [ ]:
%%time
_, feature_importance_df = kfold_lgb_rmse(train_df, test_df)

In [ ]:
%%time
_, feature_importance_df_s = kfold_lgb_rmse(train_df_s, test_df_s)

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

def objective(trial, num_folds=4, stratified=True):
    categorical_features = ['feature_1_orig', 'feature_2_orig', 'feature_3_orig','first_active_year', 'first_active_month', 'first_active_year', 'first_active_quarter']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=326)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=326)
        
    oof_preds = np.zeros(train_df_s.shape[0])
    feats = [f for f in train_df_s.columns if f not in FEATS_EXCLUDED]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df_s[feats], train_df_s['outliers'])):
        train_x, train_y = train_df_s[feats].iloc[train_idx], train_df_s['target'].iloc[train_idx]
        valid_x, valid_y = train_df_s[feats].iloc[valid_idx], train_df_s['target'].iloc[valid_idx]

        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                categorical_feature=categorical_features)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               categorical_feature=categorical_features)
        param = {'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'n_jobs': -1, 
                 'boosting_type': 'gbdt',
                 'seed':int(2**n_fold), 'drop_seed': int(2**n_fold), 'feature_fraction_seed': int(2**n_fold),
                 'learning_rate': trial.suggest_uniform('learning_rate', 1e-3, 2e-3),
                 'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 0.6),
                 'num_leaves': trial.suggest_int('num_leaves', 60, 120),
                 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 40),
                 'max_depth': trial.suggest_int('max_depth', 6,12),
                 'reg_alpha': trial.suggest_uniform('reg_alpha', 25, 50),
                 'reg_lambda': trial.suggest_uniform('reg_lambda', 10, 30),
                 'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
                 'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
                 'min_child_weight': trial.suggest_int('min_child_weight', 5, 40),
                 'min_split_gain': trial.suggest_int('min_split_gain', 5, 40),
                }
        param['bagging_freq'] = trial.suggest_int('bagging_freq', 6, 10)
        param['bagging_fraction'] = trial.suggest_uniform('bagging_fraction', 0.5, 1.0)
        param['bagging_seed'] = 11
        param['bagging_seed'] = int(2**n_fold)
    
        clf = lgb.train(param, lgb_train, 20000, valid_sets=[lgb_train, lgb_test],
                        verbose_eval=500, early_stopping_rounds=500)
            
        oof_preds[valid_idx] = clf.predict(train_df_s.iloc[valid_idx][feats], num_iteration=clf.best_iteration)
        
        del clf, train_x, train_y, valid_x, valid_y, train_idx, valid_idx
        gc.collect()
    
    return rmse(oof_preds, train_df_s['target'])


In [ ]:
%%time
study = optuna.create_study()
study.optimize(objective, n_trials=50)
print('study.best_params: \t')
print(study.best_params)
print('study.best_value: \t')
print(study.best_value)
print('study.best_trial.user_attrs: \t')
print(study.best_trial.user_attrs)

optuna_lgb = study.trials_dataframe()
optuna_lgb.to_csv("optuna_lgb_536.csv")